<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td class="header_buttons">
            <a href="FILENAME" download><img src="../../images/icons/download.png" alt="biosignalsnotebooks | download button"></a>
        </td>
        <td class="header_buttons">
            <a href="SOURCE" target="_blank"><img src="../../images/icons/program.png" alt="biosignalsnotebooks | binder server" title="Be creative and test your solutions !"></a>
        </td>
        <td></td>
        <td class="header_icons">
            <a href="../MainFiles/biosignalsnotebooks.ipynb"><img src="../../images/icons/home.png" alt="biosignalsnotebooks | home button"></a>
        </td>
        <td class="header_icons">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png" alt="biosignalsnotebooks | contacts button"></a>
        </td>
        <td class="header_icons">
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank"><img src="../../images/icons/github.png" alt="biosignalsnotebooks | github button"></a>
        </td>
        <td class="header_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_1"><div id="image_img"
        class="header_image_15"></div></td>
        <td class="header_text"> Re-Sampling of signals recorded with Android sensors</td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">Android&#9729;OpenSignals mobile&#9729;Re-Sampling</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">OpenSignals mobile application</span></strong> allows to acquire data from the sensors that are built into hardware of an android smartphone.

However, when recoding data from android sensors it has to be taken into account that the android system does not allow for recording data with a fixed sampling rate. The android system rather acquires data based on so called "sensor events". When the system picks up on one of these events it will take a sample from the sensor. These events, however, may not occur at fixed time intervals and thus result in a non-equidistant sampling.
<br>
If you prefer to have a more detailed look on how android acquires data from their sensors you can have a look at the <a href=https://developer.android.com/guide/topics/sensors/sensors_overview#java/>Android Developers guide on sensors <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>. (See the section on "Monitoring Sensor Events".)
<br><br>
In order to obtain an equidistant sampling with a fixed rate, the signal has to be re-sampled with an appropriate interpolation method. We will show how this can be achieved in this <strong><span class="color4">Jupyter notebook</span></strong> .

<hr>

<p class="steps">1 - Package imports</p>

First, lets import some useful libraries that will be used for visualization and data processing purposes.

In [3]:
# biosignalsnotebooks package
import biosignalsnotebooks as bsnb

# package in order to load .txt files
from numpy import loadtxt
import numpy as np

# package for signal interpolation
import scipy as scp

# package for plotting the data
from bokeh.plotting import figure, show

<p class="steps">2 - A look at the data as it is returned by the OpenSignals mobile app</p>

Before we begin with the re-sampling, we will have a look at the data structure that is returned from an android sensor by the OpenSignals mobile app.
For this <strong><span class="color4">notebook</span></strong> a look at the accelerometer sensor is taken. However, the procedure is the same for any android sensor type. 

In the cell below, the data is shown in its .txt file format. The file has the usual OpenSignals format, containing a header and then displaying the recorded data. From the <strong>"column"</strong> field in the header we can see that the last three columns are the x, y, and z channels of the accelerometer. Furthermore, there are two additional columns that are usually not present in files that are generated by PLUX sensors. These are the <strong>"timestamp"</strong> and the <strong>"system_time"</strong>. These indicate at what time the android system recorded the sensor event. 

Both of these columns will be needed in order to synchronize data acquired from multiple android sensors. However, this will not be part of this <strong><span class="color4">notebook</span></strong>. The synchronization procedure is shown <a href=https://developer.android.com/guide/topics/sensors/sensors_overview#java/>here (correct link still missing) <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>. 


In [2]:
# Embedding of .pdf file
from IPython.display import IFrame
IFrame(src="../../images/other/android_signal_resampling/Accelerometer.txt", width="100%", height="350")

Now lets have a look at the data in a plot. As the time axis we are going to use the <strong>"timestamp"</strong> column because it has a higher resolution. The android system records the <strong>"timestamp"</strong> in nanoseconds while the the <strong>"system_time"</strong> is recorded in milliseconds. 

For visualization purposes we will only plot the the first 100 samples of the accelerometer's z axis. Additionally, we will shift the time axis so that the recording begins at zero and we will convert it from nanoseconds to seconds

Looking at the plot that is generated, we can clearly see that the signal is not equidistantly sampled.

In [5]:
# set file path (this file path needs to be set accoridng to where you saved the data)
file_path="../../images/other/android_signal_resampling/Accelerometer.txt"

# load the data from the file
data = loadtxt(file_path)

# get timestamp column
raw_time = data[:,1]

# get the start of the acquistion
acq_start = raw_time[0]

# shift time_axis
time_axis = (raw_time - acq_start)

# convert from nanoseconds to seconds
time_axis = time_axis * 1e-9

# get x axis of accelerometer
z_acc = data[:,5]

# set the number of samples to be shown
num_samples = 100

# get the first 100 samples
time_axis_samples = time_axis[:num_samples]
z_acc_samples = z_acc[:num_samples] 

# array with zeros (for plotting the vertical lines in the plot. These are only used for visualisation purposes)
line_start = np.zeros(num_samples)

# plot the first 100 samples
p = figure()
p.segment(time_axis_samples, line_start, time_axis_samples, z_acc_samples, color="blue", line_width=1) # draw lines
p.circle(time_axis_samples, z_acc_samples, color="blue", size=10) # draw circles
p.xaxis.axis_label = 'time (s)'
p.yaxis.axis_label = 'Z-Accelerometer (m/s²)'
bsnb.opensignals_style([p]) # apply biosignalsnotebooks style
show(p)

<p class="steps">3 - Calculating the approximate sampling rate</p>

Now we will take a look at how to calculate the approximate sampling rate of the sensor. In order to do this, we will calculate the mean distance between sampling points and then calculate how many samples with this mean distance fit into one second.

As we we will see, the sensor samples on average with approximately <strong>96 Hz</strong>.

In [6]:
# calculate the distance between sampling points
sample_dist = np.diff(time_axis)

# calculate the mean distance
mean_dist = np.mean(sample_dist)

# print the mean
print('Mean distance between sampling points: {}'.format(mean_dist))

# calculate the number of samples within 1 second
samples_per_second = 1/mean_dist

# print the number of samples
print('Approximate sampling rate: {} Hz'.format(samples_per_second))

Mean distance between sampling points: 0.010418098087350041
Approximate sampling rate: 95.9868098395262 Hz


<p class="steps">4 - Re-sampling the signal</p>

Now that we have seen that the signal is not equidistantly sampled and know what is the approximate sampling rate, we can re-sample the signal to a reasonable sampling rate. <br>In this instance we are going to use a sampling rate of <strong>100 Hz</strong>.

In order to re-sample the signal, we will use one of  <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d/>scipy's interpolation methods <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>. To explain the method very briefly, it creates a function which allows for interpolation of the data to a desired range. Additionally, the interpolation method allows for setting which kind of interpolation is used. In order to keep things simple, we will just use linear interpolation, but feel free to try around a little bit with other interpolation functions.

In [7]:
# get the end of the acquistion
acq_end = time_axis[-1]

# create interpolation function
inter_func = scp.interpolate.interp1d(time_axis, z_acc, kind='linear')

# set sampling frequency
sample_freq = 100

# create new time axis
time_axis_inter = np.arange(0, acq_end, 1/sample_freq)

# interpolate the values
z_acc_inter = inter_func(time_axis_inter)

Lets plot the signal again and see the result. As we will see, the signal is now equidistantly sampled. 

In [8]:
# get the first 100 samples
time_axis_samples_inter = time_axis_inter[:num_samples]
z_acc_samples_inter = z_acc_inter[:num_samples] 

# plot the first 100 samples
p = figure()
p.segment(time_axis_samples_inter, line_start, time_axis_samples_inter, z_acc_samples_inter, color="blue", line_width=1) # draw lines
p.circle(time_axis_samples_inter, z_acc_samples_inter, color="blue", size=10) # draw circles
p.xaxis.axis_label = 'time (s)'
p.yaxis.axis_label = 'Z-Accelerometer (m/s²)'
bsnb.opensignals_style([p]) # apply biosignalsnotebooks style
show(p)

However, we need to keep in mind, that the interpolation leads to a slight shift of our data points. The shift of the points differs depending on the kind of interpolation method used.
<br>
We can see these shifts when we compare both signals in the same plot, as seen below. The original signal is shown in blue while the interpolated signal is shown in red. As before, we are only showing the first 100 samples of each signal.

Try running the code with <strong>different</strong> interpolation methods and see how the interpolated signal changes with the kind of interpolation method used.

In [9]:
p = figure()
p.line(time_axis_samples, z_acc_samples, color="blue", legend_label="original") # draw original signal
p.line(time_axis_samples_inter, z_acc_samples_inter, color="red", legend_label="interpolated") # draw interpolated signal
p.xaxis.axis_label = 'time (s)'
p.yaxis.axis_label = 'Z-Accelerometer (m/s²)'
bsnb.opensignals_style([p]) # apply biosignalsnotebooks style
show(p)

<p class="steps">5 - Writing a function that re-samples all columns</p>
Doing the interpolation manually for each column might seem a bit redundant. Thus, we will now implement a function that will do all the work for us.

The function will take the following parameters:
 <ul>
  <li><strong>time (N, array_like):</strong> A 1D array containing the original time axis of the data</li><br>
    
  <li><strong>data (...,N,..., array_like):</strong> A N-D array containing data columns that are supposed to be interpolated. The length of data along the interpolation axis has to be the same size as time.</li><br>
    
  <li><strong>start (int, optional):</strong> The sample from which the interpolation should be started. When not specified the interpolation starts at 0. When specified the signal will be cropped to this value.</li><br>
    
  <li><strong>stop (int, optional):</strong> The sample at which the interpolation should be stopped. When not specified the interpolation stops at the last value. When specified the signal will be cropped to this value.</li><br>
    
  <li><strong>shift_time_axis (bool, optional):</strong> If true the time axis will be shifted to start at zero and will be converted to seconds.</li><br>
    
  <li><strong>sampling_rate (int, optional):</strong> The sampling rate to which the signal should be re-sampled. The value should be > 0. If not specified the signal will be re-sampled to the next tens digit with respect to the approximate sampling rate of the signal (i.e. approx. sampling of 96 Hz will be re-sampled to 100 Hz).</li><br>
   <li><strong>kind_interp (string, optional):</strong> Specifies the kind of interpolation method to be used as string. If not specified, 'linear' interpolation will be used. Available options are: ‘linear’, ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’, ‘next’.</li><br>
</ul> 

In [10]:
# define function for interpolating all signal columns
def re_sample_data(time, data, start=0, stop=-1, shift_time_axis=False, sampling_rate=None, kind_interp='linear'):
    
    # crop the data and time to specified start and stop values
    if(start != 0 or stop !=-1):
        time = time[start:stop]
        
        # check for dimensionality of the data
        if(data.ndim == 1): # 1D array
            
            data = data[start:stop]
            
        else: # multidimensional array
            
            data = data[start:stop, :]
        
    # get the original time origin
    time_origin = time[0]

    # shift time axis (shifting is done in order to simplify the calculations)
    time = time - time_origin
    time = time * 1e-9
    
    # calculate the approximate sampling rate and round it to the next tens digit
    if(sampling_rate == None):
        # calculate the distance between sampling points
        sample_dist = np.diff(time)

        # calculate the mean distance
        mean_dist = np.mean(sample_dist)
        
        # calculate the sampling rate
        sampling_rate = 1/mean_dist
        
        # round it to the next tens digiit
        sampling_rate = int(np.ceil(sampling_rate / 10.0)) * 10
    
    # create new time axis
    time_inter = np.arange(time[0], time[-1], 1/sampling_rate)
    
    # check for the dimensionality of the data array.
    if(data.ndim ==1): # 1D array
        
        # create the interpolation function
        inter_func = scp.interpolate.interp1d(time, data, kind=kind_interp)
        
        # calculate the interpolated column and save it to the correct column of the data_inter array
        data_inter = inter_func(time_inter)
    
    else: # multidimensional array
        
        # create dummy array
        data_inter = np.zeros([time_inter.shape[0], data.shape[1]])
    
        # cycle over the columns of data
        for col in range(data.shape[1]):
        
            # create the interpolation function
            inter_func = scp.interpolate.interp1d(time, data[:,col], kind=kind_interp)
        
            # calculate the interpolated column and save it to the correct column of the data_inter array
            data_inter[:,col] = inter_func(time_inter)
    
    # check if time is not suppossed to be shifted
    if(not shift_time_axis):
        
        # shift back
        time_inter = time_inter * 1e9
        time_inter = time_inter + time_origin
    
    
    # return the interpolated time axis and data
    return time_inter, data_inter, sampling_rate

Now we can re-sample all three accelerometer channels at once by just calling our function. The way we call our function here will do the same re-sampling to the z axis of the accelerometer as we have done before.
<br>
This function can also be used for any other android sensor type.

In [14]:
# resample all three acceleromter channels at once (the accelerometer data are columns 3-5)
time_inter, data_inter, sampling_rate_inter = re_sample_data(raw_time, data[:,3:], shift_time_axis=True)

# plot the z-axis data for the first 100 samples
p = figure()
p.segment(time_inter[:num_samples], line_start, time_inter[:num_samples], data_inter[:num_samples, -1], color="blue", line_width=1) # draw lines
p.circle(time_inter[:num_samples], data_inter[:num_samples, -1], color="blue", size=10) # draw circles
p.xaxis.axis_label = 'time (s)'
p.yaxis.axis_label = 'Z-Accelerometer (m/s²)'
bsnb.opensignals_style([p]) # apply biosignalsnotebooks style
show(p)

In this <strong><span class="color4">Jupyter notebook</span></strong> we learned how to re-sample signals from sensors integrated into the hardware of an android smartphone and how to implement a function that conveniently handles all the re-sampling for us.

<strong><span class="color7">We hope that you have enjoyed this guide</span></strong>. <strong><span class="color2">biosiganlsnotebooks</span> <span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining</span> 
<a href=https://biosignalsplux.com/learn/notebooks.html>Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></strong>.


<hr>
<table width="100%">
    <tr>
        <td class="footer_logo">
            <img src="../../images/ost_logo.png" alt="biosignalsnotebooks | project logo [footer]">
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/aux_files/biosignalsnotebooks_presentation.pdf" target="_blank">&#9740; Project Presentation</a>
            <br>
            <a href="https://github.com/biosignalsplux/biosignalsnotebooks" target="_blank">&#9740; GitHub Repository</a>
            <br>
            <a href="https://pypi.org/project/biosignalsnotebooks/" target="_blank">&#9740; How to install biosignalsnotebooks Python package ?</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="40%" style="text-align:left">
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/biosignalsnotebooks.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="https://www.biosignalsplux.com/notebooks/Categories/MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
        </td>
    </tr>
</table>

<span class="color6"><strong>Auxiliary Code Segment (should not be replicated by
the user)</strong></span>

In [1]:

from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [11]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>